In [1]:
# name = input("Please enter your name:  (format LastName_FirstName)")
# age = int(input("Please enter your age: (Only Integer please)"))
# height = int(input("Please enter your height: (Number please in centimeters)"))
# weight = int(input("Please enter your weight: (Number please in kg)"))

name = "Bai_Muqing"
age = 27
height = 180.34
weight = 75.0
place_of_residency = "cambridge,ma,us"

In [2]:
user_info = {
    "name" : name,
    "age" : age,
    "height" : height,
    "weight" : weight,
    "place_of_residency" : place_of_residency,
}

In [3]:
import pandas as pd
df2 = pd.read_excel('230303-17_MQ_CGM.xlsx', skiprows=1,
                    sheet_name='Scanned_inputs')  # 读取scanned input sheet
df2 = df2[["Device Timestamp", "Scan Glucose mg/dL", "Notes(Calories)", "Notes(Ingredients)"]]
df2.columns = ["time", "glu", "calories", "ingredients"]
df2.head()

,time,glu,calories,ingredients
0,2023-03-03 18:45:00,111,NaN,NaN
1,2023-03-03 19:18:00,83,200.0,500ml of 8% alcoholic drinks
2,2023-03-03 20:14:00,127,NaN,NaN
3,2023-03-04 06:13:00,101,-450.0,Woke up from a 9hr sleep
4,2023-03-04 09:40:00,114,319.0,"boiled chicken breast, pan fried oyster mushro..."


In [4]:
temp = df2.dropna(subset=["ingredients", "calories"])

In [5]:
temp.head()

,time,glu,calories,ingredients
1,2023-03-03 19:18:00,83,200.0,500ml of 8% alcoholic drinks
3,2023-03-04 06:13:00,101,-450.0,Woke up from a 9hr sleep
4,2023-03-04 09:40:00,114,319.0,"boiled chicken breast, pan fried oyster mushro..."
5,2023-03-04 11:24:00,105,-62.0,took a shower
6,2023-03-04 11:50:00,116,150.0,A can of Pepsi


In [6]:
temp["time"] = temp["time"].dt.strftime("%Y-%m-%d, %H:%M:%S")

<ipython-input-6-ba3abb85b99a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["time"] = temp["time"].dt.strftime("%Y-%m-%d, %H:%M:%S")


In [7]:
temp = temp.to_dict(orient ="list")

In [8]:
user_info["eating habitat input"] = temp

In [9]:
glu_df = pd.read_csv("output.csv")
glu_df.head()

,time,glu,calories,gap_minute,glu_diff,slope
0,2023-03-03 18:32:00,120.0,NaN,0,0.0,NaN
1,2023-03-03 18:45:00,111.0,NaN,13,-9.0,-0.692308
2,2023-03-03 18:47:00,110.0,NaN,2,-1.0,-0.500000
3,2023-03-03 19:02:00,100.0,NaN,15,-10.0,-0.666667
4,2023-03-03 19:18:00,83.0,200.0,16,-17.0,-1.062500


In [10]:
glu_df = glu_df.drop(columns=["gap_minute", "glu_diff"])

In [11]:
glu_df = glu_df.fillna(0.0)
glu_df["time"] = pd.to_datetime(glu_df["time"])
glu_df["time"] = glu_df["time"].dt.strftime("%Y-%m-%d, %H:%M:%S")

In [12]:
glu_df.head()

,time,glu,calories,slope
0,"2023-03-03, 18:32:00",120.0,0.0,0.000000
1,"2023-03-03, 18:45:00",111.0,0.0,-0.692308
2,"2023-03-03, 18:47:00",110.0,0.0,-0.500000
3,"2023-03-03, 19:02:00",100.0,0.0,-0.666667
4,"2023-03-03, 19:18:00",83.0,200.0,-1.062500


In [13]:
slope = glu_df[["time", "slope"]]

In [20]:
slope = slope.fillna(0)
slope['time'] = pd.to_datetime(slope['time'])
group_slope = slope.groupby(slope["time"].dt.date)

import numpy as np

avg_slope = []
for date, group in group_slope:
    group = np.array(group['slope'])
    avg_slope.append({"date": date.strftime("%Y-%m-%d"),
                      "slope": np.absolute(group).mean()})

In [21]:
avg_slopes = [d["slope"] for d in avg_slope]

health_threshold = 0.5
# count how many values are larger than 0.5
count = sum([1 for x in avg_slopes if x > health_threshold])

# convert the float to a percentage format
health_percentage = '{:.2%}'.format(count/len(avg_slopes))

# print the result
print(health_percentage)

80.00%


In [22]:
user_info["health_slope"] = avg_slope
user_info["unhealth_ratio"] = health_percentage

In [23]:
user_info["glu_table"] = glu_df.to_dict(orient="list")

In [24]:
import json 

with open ("test.json", "w") as f:
    json.dump(user_info, f)